In [1]:
from ams.notebooks.twitter import initialize_notebook

from pathlib import Path

import pandas as pd
from pyspark.sql import DataFrame

from ams.config import constants

from ams.services import file_services

import sys
from pathlib import Path
from typing import Dict

from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, Row, StructField, DoubleType, ArrayType, StringType, FloatType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from ams.config import constants
from ams.services import file_services
from ams.services import spark_service
from ams.utils import sentiment
import findspark

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Setting up logging...
Will use logging path: C:\Users\Chris\workspaces\data\logs\alpha_media_signal


In [12]:
id_with_label_path = Path(constants.TWITTER_INFERENCE_MODEL_PATH, "twitter_id_with_label.parquet")

frac = 1.0

df_twitter = pd.read_parquet(str(id_with_label_path))[["f22_id", "buy_sell", "f22_ticker", "date"]]

In [13]:
%%time

from functools import reduce
from pyspark.sql import DataFrame
import csv

folder_path = Path(constants.TWITTER_OUTPUT_RAW_PATH, "deduped")
files = file_services.list_files(parent_path=folder_path, ends_with="parquet")

df_all = []
for f in files:
    df = pd.read_parquet(f)
    df = df[["f22_id", "nlp_text"]]
    df = df[df["f22_id"].notnull()]
    df_all.append(df)
    
df_id_fixed = pd.concat(df_all, axis=0)
df_all = None

Wall time: 2min 41s


In [14]:
# print(df_twitter[df_twitter["f22_id"] == -9223332983519753179].shape[0])
# df_sample = df_id_fixed.sample(frac=.0001)
# df_sample.sort_values(by=["f22_id"], inplace=True)
# df_sample.head(500)

In [15]:
df_id_fixed.sort_values("f22_id", inplace=True)
df_id_fixed.head(500)

,f22_id,nlp_text
1465933,-9223359071523168643,FORE @Economy_NI: New York's #cloudmanagement...
1830199,-9223359071523168643,IONA @Economy_NI: New York's #cloudmanagement...
379123,-9223357255218515402,STEN @GOPChairwoman @flyoverlady @realDonaldTr...
1725042,-9223356403058602357,NIU I had so much fun at the “Stepping Forward...
475129,-9223343210204064878,EBAY @elvis_memories: Limited Edition Pink Ca...
692716,-9223341866163119539,MANC @manc_wanderer @MissGAtkinson @petedpaint...
1529376,-9223341061937921284,REIS shu making sure reis sleep schedules in c...
832207,-9223338774945694392,DICE maybe i’ll join the next dirty dice but i...
1471115,-9223338770114220005,CREE @kamsizoe You gave up right? For how long...
955684,-9223338285735059987,SIXX @fucking_sixx this is for Dominique 👀


In [16]:
df_twitter = df_twitter.sort_values(by=["f22_id"])
df_twitter.head(20)

,f22_id,buy_sell,f22_ticker,date
1136491,-9223356403058602357,1,NIU,2020-10-15
1136490,-9223356403058602357,1,NIU,2020-10-15
873249,-9223343210204064878,1,EBAY,2020-09-25
873248,-9223343210204064878,1,EBAY,2020-09-25
60796,-9223323608447041646,-1,DOMO,2020-09-10
60797,-9223323608447041646,-1,DOMO,2020-09-10
1397537,-9223293603192513629,-1,OFS,2020-10-01
1397536,-9223293603192513629,-1,OFS,2020-10-01
798915,-9223280453642448255,1,DOMO,2020-10-12
798914,-9223280453642448255,1,DOMO,2020-10-12


In [17]:
from pyspark.storagelevel import StorageLevel

print(df_id_fixed.shape[0])
df_text_dd = df_id_fixed.drop_duplicates(subset=["f22_id"])
print(df_text_dd.shape[0])

print(df_twitter.shape[0])
df_twitter_dd = df_twitter.drop_duplicates(subset=["f22_id"])
print(df_twitter_dd.shape[0])

8064206
5354378
1930904
949885


In [18]:
df_joined = pd.merge(df_twitter_dd, df_text_dd, on="f22_id", suffixes=["", "_right"])
df_joined.shape[0]

949885

In [19]:
print(df_joined.columns)
df_joined.head(10)

Index(['f22_id', 'buy_sell', 'f22_ticker', 'date', 'nlp_text'], dtype='object')


,f22_id,buy_sell,f22_ticker,date,nlp_text
0,-9223356403058602357,1,NIU,2020-10-15,NIU I had so much fun at the “Stepping Forward...
1,-9223343210204064878,1,EBAY,2020-09-25,EBAY @elvis_memories: Limited Edition Pink Ca...
2,-9223323608447041646,-1,DOMO,2020-09-10,DOMO Domo mea 🛣
3,-9223293603192513629,-1,OFS,2020-10-01,OFS @dfobbsr_ofs @DGFS_HGs_CD @cfo_so_fs @focr...
4,-9223280453642448255,1,DOMO,2020-10-12,DOMO Shop https://t.co/MWavrYzbyv for your cus...
5,-9223247829049314849,-1,ACER,2020-08-17,ACER To one of the greatest captains of our ti...
6,-9223247824113695041,-1,DOMO,2020-10-07,DOMO @Mr_Cwilly @Domo_LXXXVI They treated doug...
7,-9223227950643901747,-1,EBAY,2020-10-05,EBAY @DeSplainsCards What are they charging on...
8,-9223223559497645690,1,MGPI,2020-10-16,MGPI MGP Ingredients Schedules Third Quarter 2...
9,-9223218594977255735,-1,LOOP,2020-10-07,LOOP @PE_loser: Worship BBC&#44; sissies. Acc...


In [20]:
from pyspark.sql.functions import expr

def new_buy_sell(buy_sell):
    return 0 if buy_sell == -1 else buy_sell


df_joined["buy_sell"] = df_joined["buy_sell"].apply(new_buy_sell)
df_correct_sell = df_joined[["f22_id", "nlp_text", "buy_sell", "date"]]

In [21]:
df_correct_sell.head(10)

,f22_id,nlp_text,buy_sell,date
0,-9223356403058602357,NIU I had so much fun at the “Stepping Forward...,1,2020-10-15
1,-9223343210204064878,EBAY @elvis_memories: Limited Edition Pink Ca...,1,2020-09-25
2,-9223323608447041646,DOMO Domo mea 🛣,0,2020-09-10
3,-9223293603192513629,OFS @dfobbsr_ofs @DGFS_HGs_CD @cfo_so_fs @focr...,0,2020-10-01
4,-9223280453642448255,DOMO Shop https://t.co/MWavrYzbyv for your cus...,1,2020-10-12
5,-9223247829049314849,ACER To one of the greatest captains of our ti...,0,2020-08-17
6,-9223247824113695041,DOMO @Mr_Cwilly @Domo_LXXXVI They treated doug...,0,2020-10-07
7,-9223227950643901747,EBAY @DeSplainsCards What are they charging on...,0,2020-10-05
8,-9223223559497645690,MGPI MGP Ingredients Schedules Third Quarter 2...,1,2020-10-16
9,-9223218594977255735,LOOP @PE_loser: Worship BBC&#44; sissies. Acc...,0,2020-10-07


In [22]:
from ams.services import dataframe_services

parent_path = Path(constants.OVERFLOW_DATA_PATH, "twitter", "inference_model_drop")

tag = 'twitter_text_with_proper_labels'

file_path = Path(parent_path, f"{tag}.parquet" )

df_correct_sell.to_parquet(file_path)

file_path

WindowsPath('D:/workspaces/data/twitter/inference_model_drop/twitter_text_with_proper_labels.parquet')